## Building your Deep Neural Network: step by step

Welcome to your week4 assignemnt(part 1 of 2)! Previously you trained 2-layer Nearual Network with a single hidden layer. This week, you will build a deep neural network with as many layers as you want. 
- in this notebook, you'll implement all the functions required to build a deep neural nework.
- For the next assignment, you'll use these functions to build a deep neural network for image classification. 

#### By the end of this assignement, you'll be able to: 
- Use non-linear units like RelU to improve your model. 
- build  a deeper neural network (with more than 1 hidden layer)
- implement an easy-to-use neural network class



### 1 - Packages

First, import all the packages you'll need during this assignment.  
- numpy 
- matplotlib: is a library to plot graphs in Python
- dnn_utils provides some necessary funcitons for this notebook
- testCases provides soome test cases to assess to connectness of your functions
- np.random.seed(1) is used to keep all the random function calls consistent.  It helps grade your work. please don't change the seed!



In [9]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

from testCases import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward
from public_tests import *

import copy

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

### 2 -Outline

To build your neural network, you'll be implementing several 'helper functions'. There helper functions will be used in the next assignement to build a two-layer neural network and an L-layer neural network. 

Each small helper function will have detailed instructions to walk you through the necessary steps. Here is an outline of the steps in this assignemnt. 
- Initialize the parameters for a two-layer network and for an L-layer neural network. 
- Implement the forward propagation module (shown in the purple in the figure below)
    - Complete the LINEAR part of a layer's forward propagation step (resulting in Z)
    - The ACTIVATION function is provided for you (relu, sigmoid)
    - Combine the previous two steps into a new forward function
    - Stack the forward function L-1 time and add a sigmoid at the end. This give you a new L_model_forward function.  
-  Complete the loss
- Implement the backword propagation module 
- Finally, update the parameters



### 3 - Initialization

You will write two helper functions to initialize the parameters for your model. The first function will be used to initialize parameters for a two layer model.  The second one generalizes this initialization process to L layers. 


### 3.1 - 2-layer Neural Network 

#### Exercise 1 - initialize_parameters
Create and initialize the parameters of the 2-layer neural nework. 

**Instructions** 
- The model's structure is: LINEAR -> RELU -> LINEAR -> SIGMOID
- Use this random initialization for the weight matrics
- Use zero initialization for the biases


In [10]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument: 
    n_x: size of the input layer 
    n_h: size of hidden layer
    n_y: size of output layer
    
    Returns:
    Parameters: python dictionary containing your parameters:
        W1 -- weight matrix of shape (n_h, n_x)
        b1 -- bias vector of shape (n_h, 1)
        W2 - weight matrix of shape (n_y, n_h)
        b2 - bias vector of shape (n_y, 1)
    """
    np.random.seed(1)
    #(≈ 4 lines of code)
    # W1 = ...
    # b1 = ...
    # W2 = ...
    # b2 = ...
    # YOUR CODE STARTS HERE
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    # YOUR CODE ENDS HERE
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    
    

In [11]:
print("Test Case 1:\n")
parameters = initialize_parameters(3,2,1)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_test_1(initialize_parameters)

print("\033[90m\nTest Case 2:\n")
parameters = initialize_parameters(4,3,2)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_test_2(initialize_parameters)

Test Case 1:

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]
 All tests passed.

Test Case 2:

W1 = [[ 0.01624345 -0.00611756 -0.00528172 -0.01072969]
 [ 0.00865408 -0.02301539  0.01744812 -0.00761207]
 [ 0.00319039 -0.0024937   0.01462108 -0.02060141]]
b1 = [[0.]
 [0.]
 [0.]]
W2 = [[-0.00322417 -0.00384054  0.01133769]
 [-0.01099891 -0.00172428 -0.00877858]]
b2 = [[0.]
 [0.]]
 All tests passed.


### 3.2 L-layer Neural Network

The initialization for a deeper L-layer neural network is more complicated because there are many more weight matrices and bias vectors. 
When completing the initialize_parameters_deep function, you should make sure that your demensions match between each layer.  
For example, if the size of your input X is (12288, 209) (with m = 209 examples) then: 

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> <b>Shape of W</b> </td> 
        <td> <b>Shape of b</b>  </td> 
        <td> <b>Activation</b> </td>
        <td> <b>Shape of Activation</b> </td> 
    <tr>
    <tr>
        <td> <b>Layer 1</b> </td> 
        <td> $$(n^{[1]},12288)$$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> <b>Layer 2</b> </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>  
   <tr>
       <td> <b>Layer L-1</b> </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
   <tr>
   <tr>
       <td> <b>Layer L</b> </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>



Remember that when you compute $W X + b $ in python, it carries out broadcasting. for example, if :


$$ W = \begin{bmatrix}
    w_{00}  & w_{01} & w_{02} \\
    w_{10}  & w_{11} & w_{12} \\
    w_{20}  & w_{21} & w_{22} 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    x_{00}  & x_{01} & x_{02} \\
    x_{10}  & x_{11} & x_{12} \\
    x_{20}  & x_{21} & x_{22} 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    b_0  \\
    b_1  \\
    b_2
\end{bmatrix}\tag{2}$$

Then $WX + b$ will be:

$$ WX + b = \begin{bmatrix}
    (w_{00}x_{00} + w_{01}x_{10} + w_{02}x_{20}) + b_0 & (w_{00}x_{01} + w_{01}x_{11} + w_{02}x_{21}) + b_0 & \cdots \\
    (w_{10}x_{00} + w_{11}x_{10} + w_{12}x_{20}) + b_1 & (w_{10}x_{01} + w_{11}x_{11} + w_{12}x_{21}) + b_1 & \cdots \\
    (w_{20}x_{00} + w_{21}x_{10} + w_{22}x_{20}) + b_2 &  (w_{20}x_{01} + w_{21}x_{11} + w_{22}x_{21}) + b_2 & \cdots
\end{bmatrix}\tag{3}  $$




#### Exercise 2 - initialize_parameters_deep

Implement initialization for an L-layer Neural Network

**Instructions**
- The model's structure is [LINEAR -> RELUE] X (L-1) -> LINEAR -> SIGMOID. I.e., It has L-1 layers using RELU activation funcition followed by an output layer with a sigmoid activation funtion. 
- Use random initialization for the weight matrics. Use np.random.randon(d0,d1,..., dn) * 0.01
- Use zeros initialization for the biases
- You'll store $n^l$, the number of units in different layers, in a variable layer_dims. Now you will generalize this to L layers. 
- Here is the implementation for L = 1. It should inspire you to implement the general case

if L == 1:
    parameters["W" + str(L)] = np.randon.randn(layer_dims[1], layer_dims[0]) * 0.01
    parameters["b" + str(L)] = np.zeros((layer_dims[1],1))
    

In [20]:
## GRADE FUNCTION: initialize_parameters_deep
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python arrary (list) containing the dimensions of each layer in out network
    returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
        wl -- weight matrix of shape (layer_dims[l], layer_dim[l-1])
        bl -- bias vector of shape (layer_dims[l], 1)
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        ## YOUR CODE STARTS HERE
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        ## YOUR CODE ENDS HERE
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l],1))
    return parameters

In [22]:
print("Test Case 1 ")
parameters = initialize_parameters_deep([5,4,3])

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_deep_test_1(initialize_parameters_deep)

Test Case 1 
W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]
 All tests passed.


### 4 - Forward Propagation Module

#### 4.1 - Linear Forward
Now that you have initialized your parameters, you can do the forward propagation modules. Start by implementing some basic functions that you can use again later when implementing the model. Now, you'll complete three functions in this order: 
- LINEAR 
- LINEAR -> ACTIVATION where ACTICATION will be either ReLU or Sigmoid
- [LINEAR -> RELU] x (L-1) -> LINEAR -> SIGMOID (whole model)

The linear forward module (vectorized over all the examples) computes the following equations:

$$
Z^{[l]} = W^{[l]}A^{[l]} + b^{[l]} \tag{4}
$$

where $A^{[0]} = X$



#### Exercise 3 - linear_forward

Build the linear part of forward propagation.
 
**Reminder** The mathematical representation of this unit is $Z^{[l]} = W^{[l]}A{[l-1]} + b^{[l]}. You may also find np.dot() useful. if your dimensions don't match. printing W.shpae may help.



In [24]:
## GRADE FUNCTION: linear_forward 
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation. 
    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    
    Returns:
    Z -- the input of the activation function, also called pre-activation parameter
    cache -- a python tuple containing "A", "W" and "b"; stored for computing the backward pass efficiently. 
    
    """
    ## YOUR CODE STARTS HERE
    Z = np.dot(W, A) + b 
    
    ## YOUR CODE ENDS HERE
    
    cache = (A, W, b)
    
    return Z, cache

In [30]:
t_A, t_W, t_b = linear_forward_test_case()
print("Z = " + str(t_Z))
linear_forward_test(linear_forward)

Z = [[ 3.26295337 -1.23429987]]
 All tests passed.


### 4.2 - Linear-Activation Forward

In this notebook, you will use two activation functions:

-  **Sigmoid**:  $\sigma(Z) = \sigma(W A + b) = \frac{1}{1+ e^{-W A +b}}$. You've been provided with the `sigmoid` function which returns **two** items: the activation value `a` and a `cache` that contains `Z` (It's what we will feed in to the corresponding backword function). To use it you could just call:
```python
A, activation_cache = sigmoid(Z)

````
- **ReLU**: The mathematical formula for ReLu is $A = RELU(Z) = max(0, Z)$. You've been provided with the `relu` function. This function returns **two** items: the activation value `A` and a `cache` that contains `Z` (it's what you'll feed in to the corresponding backward function). To use it you could just call:

``` python
A, activation_cache = relu(Z)
```

For added convenience, you're going to group two functions into one function. Hence, you'll implement a function that does the LINEAR forward step followd by an ACTIVATION forward step.



#### Exercise 4 - Linear_activation_forward
Implement the forward propagation of the *LINEAR -> ACTIVATION layer. 

In [36]:
## GRADE FUNCTION: linear_activation_forward
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
    """
    if activation == 'sigmoid':
        #(≈ 2 lines of code)
        # Z, linear_cache = ...
        # A, activation_cache = ...
        # YOUR CODE STARTS HERE

        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    
    # YOUR CODE ENDS HERE   
    elif activation == 'relu':
        
        #(≈ 2 lines of code)
        # Z, linear_cache = ...
        # A, activation_cache = ...
        # YOUR CODE STARTS HERE
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)


        # YOUR CODE ENDS HERE


        
    cache = (linear_cache, activation_cache)
    
    return A, cache
    

In [37]:
t_A_prev, t_W, t_b = linear_activation_forward_test_case()

t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "sigmoid")
print("With sigmoid: A = " + str(t_A))

t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "relu")
print("With ReLU: A = " + str(t_A))

linear_activation_forward_test(linear_activation_forward)

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]
 All tests passed.


**Note**: In deep learning, the "LINEAR -> ACTIVATION" computation is counted as a single layer in the neural network, not two layers. 

### 4.3 - L-Layer Model

For even more convenience when implementing the L-layer Neural Net, you will need a function that replicates one (linear_activation_forward with RELU) L-1 times, then follows that with one linear_activation_forward with sigmoid. 


### Exercise 5 - L_model_forward
**Instructions**: In the code below, the variable `AL` will denote $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$. (This is sometimes also called `Yhat`, i.e., this is $\hat{Y}$.) 

**Hints**:
- Use the functions you've previously written
- Use for loop to replicate [LINEAR->RELU] (L-1) times
- Don't forget to keep track of the caches in the "caches" list. TO ad a new value c to a list , you can use list.append(c)


In [55]:
## GRADE FUNCTION: L_model_forward
def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- activation value from the output (last) layer
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """
    caches = []
    A = X
    L = len(parameters) // 2
    
    ## Implement LINEAR -> RELU  * (L -1). Add cache to the caches list
    ## The for loop starts at 1 beacuse layer 0 is the input
    for l in range(1, L - 1):
        print(l)
        A_prev = A
        
        #(≈ 2 lines of code)
        # A, cache = ...
        # caches ...
        # YOUR CODE STARTS HERE
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], 'relu')
        caches.append(cache)
        
        # YOUR CODE ENDS HERE
    
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    #(≈ 2 lines of code)
    # AL, cache = ...
    # caches ...
    # YOUR CODE STARTS HERE
    AL, cache = linear_activation_forward(A_prev, parameters['W' + str(L)], parameters['b' + str(L)], 'sigmoid')
    caches.append(cache)
    
    # YOUR CODE ENDS HERE
          
    return AL, caches
    

In [56]:
t_X, t_parameters = L_model_forward_test_case_2hidden()


In [57]:
t_AL, t_caches = L_model_forward(t_X, t_parameters)


1


ValueError: shapes (1,3) and (5,4) not aligned: 3 (dim 1) != 5 (dim 0)

In [46]:
t_parameters

{'W1': array([[ 0.35480861,  1.81259031, -1.3564758 , -0.46363197,  0.82465384],
        [-1.17643148,  1.56448966,  0.71270509, -0.1810066 ,  0.53419953],
        [-0.58661296, -1.48185327,  0.85724762,  0.94309899,  0.11444143],
        [-0.02195668, -2.12714455, -0.83440747, -0.46550831,  0.23371059]]),
 'b1': array([[ 1.38503523],
        [-0.51962709],
        [-0.78015214],
        [ 0.95560959]]),
 'W2': array([[-0.12673638, -1.36861282,  1.21848065, -0.85750144],
        [-0.56147088, -1.0335199 ,  0.35877096,  1.07368134],
        [-0.37550472,  0.39636757, -0.47144628,  2.33660781]]),
 'b2': array([[ 1.50278553],
        [-0.59545972],
        [ 0.52834106]]),
 'W3': array([[ 0.9398248 ,  0.42628539, -0.75815703]]),
 'b3': array([[-0.16236698]])}